# Clasificación para prescripción de medicamentos

En el dataset "drug200.csv" se encuentran los datos de medicamentos recetados para una determinada patología en función de las características de cada paciente.

## Carga y análisis de datos

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline

df_drug = pd.read_csv("data/drug200.csv")

In [11]:
print(df_drug.head())

   Age Sex      BP Cholesterol  Na_to_K   Drug
0   23   F    HIGH        HIGH   25.355  DrugY
1   47   M     LOW        HIGH   13.093  drugC
2   47   M     LOW        HIGH   10.114  drugC
3   28   F  NORMAL        HIGH    7.798  drugX
4   61   F     LOW        HIGH   18.043  DrugY


In [12]:
print(df_drug.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB
None


In [13]:
df_drug['Drug'].value_counts()

Drug
DrugY    91
drugX    54
drugA    23
drugC    16
drugB    16
Name: count, dtype: int64

In [14]:
print(df_drug.isnull().sum())

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64


Podemos ver cláramente que se trata de un **problema de clasificación multiclase**, ya que la variable objetivo "Drug" tiene 5 clases diferentes.
El dataset tiene **variables categóricas** que deberán ser transformadas para poder ser utilizadas en los algoritmos de clasificación.
Comprobamos que no hay valores nulos en el dataset.

## Separación de datos

In [15]:
X = df_drug.drop(columns=["Drug"])
y = df_drug["Drug"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

En clasificaciones multiclase es habitual muestrear estratificando para garantizar que haya muestras de todas las clases en ambos conjuntos. Utilizamos el parámetro `stratify` de la función `train_test_split` para garantizarlo, pasando la variable objetivo como argumento.

## Procesado de columnas categóricas

In [16]:
print(X_train['Sex'].value_counts())
print(X_train['BP'].value_counts())
print(X_train['Cholesterol'].value_counts())

Sex
M    85
F    75
Name: count, dtype: int64
BP
HIGH      63
LOW       50
NORMAL    47
Name: count, dtype: int64
Cholesterol
NORMAL    82
HIGH      78
Name: count, dtype: int64


Vemos que tenemos dos columnas binarias y una columna con 3 categorías donde el orden es importante.
<!-- TODO: Dentro de las binarias Sex no tiene orden y Cholesterol sí. Hay un cierto debate sobre si tiene sentido OHE sacando dos columnas invertidas de Sex según el algoritmo (¿en redes neuronales?), aunque son totalmente correladas. -->

In [17]:
X_train_tr = X_train.copy()

X_train_tr['Sex'] = X_train['Sex'].map({'M': 0, 'F': 1})
X_train_tr['BP'] = X_train['BP'].map({'LOW': 0, 'NORMAL': 1, 'HIGH': 2}) # Importante el orden
X_train_tr['Cholesterol'] = X_train['Cholesterol'].map({'NORMAL': 0, 'HIGH': 1})

## Varios modelos de clasificación

In [18]:
# Regresión logística
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=10000).fit(X_train_tr, y_train)

# Árbol de decisión
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X_train_tr,y_train)

# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=14).fit(X_train_tr, y_train)

# SVM
from sklearn.svm import SVC
svc = SVC().fit(X_train_tr, y_train)

# Naive Bayes
from sklearn.naive_bayes import CategoricalNB
nb = CategoricalNB().fit(X_train_tr, y_train)

# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_leaf_nodes=30).fit(X_train_tr, y_train)

# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train_tr, y_train)

## Comparación de modelos

### Transformación del conjunto de test

Para testear el modelo, necesitamos seguir el mismo *pipeline* que hemos aplicado al conjunto de entrenamiento. Es necesario por tanto realizar las mismas transformaciones de las columnas categóricas.

In [19]:
X_test_tr = X_test.copy()
X_test_tr['Sex'] = X_test['Sex'].map({'M': 0, 'F': 1})
X_test_tr['BP'] = X_test['BP'].map({'LOW': 0, 'NORMAL': 1, 'HIGH': 2})
X_test_tr['Cholesterol'] = X_test['Cholesterol'].map({'NORMAL': 0, 'HIGH': 1})

### Exactitud (*acccuracy*)

In [41]:
print("Exactitud de la regresión logística:")
y_pred_log_reg = log_reg.predict(X_test_tr)
print("     con cálculo manual: ", np.sum(y_pred_log_reg==y_test)/len(y_test))
print("     con función 'accuracy_score': ", accuracy_score(y_test, y_pred_log_reg))
print("     con método 'score': ", log_reg.score(X_test_tr, y_test))
print()


pd.DataFrame( # Tabla de exactitudes ordenadas
    {'Exactitud': {
        'Logistic Regression': log_reg.score(X_test_tr, y_test),
        'K Neighbors': knn.score(X_test_tr, y_test),
        'SVM': svc.score(X_test_tr, y_test),
        'Categorical NB': nb.score(X_test_tr, y_test),
        'Decision Tree': tree.score(X_test_tr, y_test),
        'Random Forest': rf.score(X_test_tr, y_test),
        'Gaussian NB': gnb.score(X_test_tr, y_test)
        }
     }).sort_values(by='Exactitud', ascending=False)

Exactitud de la regresión logística:
     con cálculo manual:  0.975
     con función 'accuracy_score':  0.975
     con método 'score':  0.975



,Exactitud
Logistic Regression,0.975
Random Forest,0.975
Decision Tree,0.975
Gaussian NB,0.900
Categorical NB,0.850
SVM,0.700
K Neighbors,0.700


### Matriz de confusión y *classification report*

In [26]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_log_reg))

[[18  0  0  0  0]
 [ 0  5  0  0  0]
 [ 0  0  3  0  0]
 [ 0  0  0  3  0]
 [ 1  0  0  0 10]]


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

       DrugY       0.95      1.00      0.97        18
       drugA       1.00      1.00      1.00         5
       drugB       1.00      1.00      1.00         3
       drugC       1.00      1.00      1.00         3
       drugX       1.00      0.91      0.95        11

    accuracy                           0.97        40
   macro avg       0.99      0.98      0.99        40
weighted avg       0.98      0.97      0.97        40



## Pipeline

Una de las utilidades de la clase `Pipeline` es precisamente dejar definido un ojeto que representa todos los pasos a seguir para así poder aplicarlos fácilmente a los datos de testing, evitando depender de modificar un script, como hicimos arriba, que sería más susceptible a errores (hacer algún paso distinto).

In [22]:
ordinal_transformer = ColumnTransformer(
    transformers=[
        ('sex', OrdinalEncoder(), ['Sex']),
        ('bp', OrdinalEncoder(categories=[['LOW', 'NORMAL', 'HIGH']]), ['BP']), # Especificamos el orden
        ('cholesterol', OrdinalEncoder(), ['Cholesterol'])
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(ordinal_transformer, DecisionTreeClassifier()) 
model = pipeline.fit(X_train, y_train)
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('sex', OrdinalEncoder(),
                                                  ['Sex']),
                                                 ('bp',
                                                  OrdinalEncoder(categories=[['LOW',
                                                                              'NORMAL',
                                                                              'HIGH']]),
                                                  ['BP']),
                                                 ('cholesterol',
                                                  OrdinalEncoder(),
                                                  ['Cholesterol'])])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [23]:
model.score(X_test, y_test)

0.975

## Fuente

- [Drug Classification dataset](https://www.kaggle.com/prathamtripathi/drug-classification)